### Updating The FX_derivatives_all database 
Update with the TJ06 and TX06sap reports.

In [2]:
#Imports the daily TJ06 (options) & TX06 (forwards) and updates FX_Derivatives_All Database 
#which is located in the Financial_Database directory located within the Treasury drive

import pandas as pd
import csv
import datetime
import openpyxl

#this function module returns the delta days for file date (ie. If today is Monday, we need Friday's date
#for the SAP file)
def whatday():
    wkday = datetime.datetime.today().weekday()
    if wkday == 0:    
        return 3
    else:
        return 1

#this function module converts floats to strings.
def change2str(df, column):
    df[column] = df[column].astype(int)
    df[column] = df[column].astype(str)

#this function inverts JPY rate    
def buy_calc(rate,currency):
    if currency == 'JPY':
        buy_rate = 1/rate
    else:
        buy_rate = rate
    return buy_rate

#this function returns the start & end cell references
def table_range(df):
    row, col = df.shape
    end_cell = f'{chr(col + 65)}{row + 1}'
    return 'A1:' + end_cell
    
    
#Putting into a variable date to be used with the file names
today = datetime.date.today()
prevday = today - datetime.timedelta(days=whatday())
prevday = prevday.strftime('%m%d%Y')

#Build file paths to get the daily TJ06 (options) and TX06 (forwards) files. 
dir_path = r'\\or1isi1vl3921nfs\sapnfsc\PRD003\0256'
file1_name = '\TX06_Extract' + prevday + '.txt'
file2_name = '\TJ06_Extract' + prevday + '.txt'
file_path1 = dir_path + file1_name
file_path2 = dir_path + file2_name

#Formating the header for daily TJ06 (options) and TX06 (forwards) files.
df1 = pd.read_csv(file_path1, delimiter = '|', skiprows=[0,2], header =[0])
df2 = pd.read_csv(file_path2, delimiter = '|', header =[0])

#Aligned header to match data columns for TJ06 (Option) report and removed blank spaces on header fields.
names = list(df2.columns.values)
newname = []
for name in names:
    newname.append(name.strip())
newname.append(newname.pop(0))
df2.columns = newname
df2.reset_index()


df2.insert(0,"Entity", 'ADIR')
df2.dropna(subset = ["Transactn"], inplace=True)

df2.reset_index(drop=True, inplace=True)

df2.dropna(axis=1, how='all', inplace=True)

#create dictionary of counter party names and their associated numbers
counter_party_dicts = {'61':'JPM',
                       '2':'BOA',
                       '2081':'USB',
                       '2121':'WFB',
                       '2101':'HSBC',
                       '12600': 'SG',
                       '1200000141': 'BOA',
                       '2131':'MSC',
                       '3':'UBS',
                       '5':'GS',
                       '72':'CITI',
                       '10020':'RBS'}

#This changes format by using the change2str function.
change2str(df2, 'Counterparty')
change2str(df2, 'Transactn')   

#This converts the SAP BP code to a counterparty name.
i=0
for counterparties in df2.loc[:,'Counterparty']:
    
    df2.loc[i,'Counterparty'] = counter_party_dicts[counterparties]
    i+=1

#Creating dictionary to add currency pair
currencypair_dict = {'EUR':'EURUSD',
                    'GBP':'GBPUSD',
                    'AUD':'AUDUSD',
                    'JPY':'USDJPY',
                    'INR':'USDINR'}    

#Add additional columns
df2.insert(0, 'ID',"ADIR" + df2['Transactn'])

#Removing unwanted columns
df2.drop(columns=["Type", "Exercise", "Settlement", "Premium Curr", "Transactn"], inplace= True)



#Add additional columns
df2['Currency_Pair'], df2['Buy_Currency'], df2['Sell_Currency'], df2['Buy_Amount'] = ["", "USD", "", 0]

#Split underlying column into sell amount and sell currency amount.
for index, amount in enumerate(df2['Underlying']):
    Curr, Amt = amount.strip().split(' ')
    
    df2.loc[index,'Sell_Currency'] = Curr
    df2.loc[index,'Currency_Pair'] = currencypair_dict[Curr]
    
#Adding Buy Amount buy multiplying the strike and underlying. Also trancating the Buy_Amount to two decimals
for index, amount in enumerate(df2['Underlying']):
    Curr, Amt = amount.strip().split(' ')
    Amt = float(Amt.replace(',','').strip('-'))
    df2.loc[index,'Buy_Amount'] = Amt*buy_calc(df2.loc[index,'Strike'], Curr)
    df2.loc[index,'Underlying'] = Amt*-1
    
#Rename columns to match FX Derivatives all
df2.rename(columns = {"Premium Amt":"Premium", 
                      "Term Start":"Trade_Date", 
                      "Due Date":"Expiry_Date", 
                      "Strike":"Contract_Rate",
                      "Underlying":"Sell_Amount"},inplace=True)


#reordering column names
df2 = df2[['ID',
           'Entity',
           'Counterparty',
           'Currency_Pair',
           'Buy_Currency',
           'Sell_Currency',
           'Trade_Date',
           'Expiry_Date',
           'Contract_Rate',
           'Buy_Amount',
           'Sell_Amount',
           'Premium']]

df2.set_index('ID', inplace=True)

#Open test database
file_name = r'\\sjshare\finance\Treasury\Financial_Database\test\FX_Derivatives_Alltest.xlsx'
workbook = openpyxl.load_workbook(file_name)
sheet = workbook['FX_options_static'] # Get a sheet from the workbook.
last_row = sheet.max_row
next_row = last_row + 1

#get a listing of the ids from the spreadsheet
id_nos = []
for ids in range ( 2 , next_row):
    id_nos.append(sheet.cell(row=ids, column=1).value)
df3 = df2[(~df2.index.isin(id_nos))]

df3['Trade_Date'] = df3['Trade_Date'].apply(lambda x: pd.Timestamp(x))
df3['Expiry_Date'] = df3['Expiry_Date'].apply(lambda x: pd.Timestamp(x))
df3['Sell_Amount'] = df3['Sell_Amount'].astype('int64')
df3['Premium'] = df3['Premium'].str.replace(',','').str.strip().astype('float')

df_old = pd.read_excel(file_name, sheet_name='FX_options_static')
df_old


df_old.set_index('ID', inplace=True)

df_new = df_old.append(df3)
df_new

C:\Users\mnunes\Anaconda3\lib\site-packages\ipykernel_launcher.py:159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\mnunes\Anaconda3\lib\site-packages\ipykernel_launcher.py:160: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\mnunes\Anaconda3\lib\site-packages\ipykernel_launcher.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.or

,Entity,Counterparty,Currency_Pair,Buy_Currency,Sell_Currency,Trade_Date,Expiry_Date,Contract_Rate,Buy_Amount,Sell_Amount,Premium,Sold_to_Rev,Sold_to_PL,Sold_Date
ID,,,,,,,,,,,,,,
ADIR3601,ADIR,GS,EURUSD,USD,EUR,2012-05-08,2012-10-29,1.3035,1.303500e+07,-10000000,408600.00,66087.0,0.0,00:00:00
ADIR3636,ADIR,JPM,GBPUSD,USD,GBP,2012-05-29,2012-10-30,1.5610,7.805000e+06,-5000000,189500.00,0.0,0.0,00:00:00
ADIR3602,ADIR,GS,USDJPY,USD,JPY,2012-05-08,2012-11-02,79.6500,1.255493e+07,-1000000000,340500.00,75900.0,0.0,00:00:00
ADIR3617,ADIR,UBS,EURUSD,USD,EUR,2012-05-15,2012-11-07,1.2785,1.278500e+07,-10000000,443500.00,0.0,22700.0,00:00:00
ADIR3623,ADIR,UBS,EURUSD,USD,EUR,2012-05-23,2012-11-14,1.2590,1.259000e+07,-10000000,466000.00,0.0,7554.0,00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ADIR8277,ADIR,HSBC,GBPUSD,USD,GBP,2020-09-29,2021-04-22,1.2852,1.542240e+07,-12000000,493057.39,NaN,NaN,NaN
ADIR8278,ADIR,BOA,USDJPY,USD,JPY,2020-09-29,2021-04-22,105.3300,3.133011e+07,-3300000000,729338.00,NaN,NaN,NaN
ADIR8288,ADIR,USB,EURUSD,USD,EUR,2020-10-06,2021-05-06,1.1842,2.723660e+07,-23000000,589950.00,NaN,NaN,NaN


In [17]:
#overwrite worksheet in the workbook.
#write a table not just dump data in spreadsheet.

workbook = openpyxl.load_workbook(r'\\sjshare\finance\Treasury\Financial_Database\test\FX_Derivatives_Alltest.xlsx')
sheet = workbook['FX_options_static'] # Get a sheet from the workbook.

filename = r'\\sjshare\finance\Treasury\Financial_Database\test\testingit5.xlsx'
sheetname = 'FX_options_static'
with pd.ExcelWriter(filename) as writer:
    if not df_new.index.name:
        df_new.index.name = 'Index'
    df_new.to_excel(writer, sheet_name=sheetname)
wb = openpyxl.load_workbook(filename = filename)

df3['Trade_Date'] = df3['Trade_Date'].apply(lambda x: pd.Timestamp(x))
df3['Expiry_Date'] = df3['Expiry_Date'].apply(lambda x: pd.Timestamp(x))
df3['Sell_Amount'] = df3['Sell_Amount'].astype('int64')
df3['Premium'] = df3['Premium'].str.replace(',','').str.strip().astype('float')

#tab = openpyxl.worksheet.table.Table(displayName=sheet._tables[0].name, ref=f'A1:{chr(len(df_new.columns)+64)}{len(df_new)+1}')
tab = openpyxl.worksheet.table.Table(displayName=sheet._tables[0].name, ref=table_range(df_new))



wb[sheetname].add_table(tab)
wb.save(filename)

In [ ]:
filename.info()

In [ ]:
sheet._tables[0].name

In [ ]:
tab = openpyxl.worksheet.table.Table(displayName=sheet._tables[0].name, ref=table_range(df_new))

In [16]:
table_range(df_new)

'A1:O800'

In [13]:
 end_cell = f'{chr(col + 65)}{row + 1}'

NameError: name 'col' is not defined